In [1]:
import numpy as np
import pandas as pd

In [171]:
df1 = pd.read_csv('company.csv')
df2 = pd.read_csv('company_data.csv')

In [3]:
df1.head()

,证券代码,日期
0,#000007,2014
1,#000403,2015
2,#000408,2016
3,#000408,2017
4,#000426,2015


In [4]:
df2.head()

,证券代码,日期,收入类型,收入额
0,1,2008/12/31,1,1.084218e+10
1,1,2008/12/31,2,1.259789e+10
2,1,2008/12/31,3,1.451312e+10
3,1,2008/12/31,4,1.063843e+09
4,1,2008/12/31,5,8.513880e+08


In [172]:
#删除df2中空值及负值
df2.loc[df2['收入额']<0,'收入额'] = np.nan
df2.dropna(inplace=True)

# 将df2中的证券代码补0至六位，并在最前面加上#
df2['证券代码'] = df2['证券代码'].map(lambda x:'#' + str(x).zfill(6))

# 保留df2中日期的前四位，并将它转为int格式
df2['日期'] = df2['日期'].str[0:4].astype('int')
# 转换也可以用 df2['日期'] = df2['日期'].str[0:4].apply(lambda x:int(x))

In [156]:
df2.head()

,证券代码,日期,收入类型,收入额
0,#000001,2008,1,1.084218e+10
1,#000001,2008,2,1.259789e+10
2,#000001,2008,3,1.451312e+10
3,#000001,2008,4,1.063843e+09
4,#000001,2008,5,8.513880e+08


In [7]:
df1.info()  # 检查df1和df2的格式是否相同，后续进行连接

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048 entries, 0 to 1047
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   证券代码    1048 non-null   object
 1   日期      1048 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 16.5+ KB


In [8]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 940667 entries, 0 to 964021
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   证券代码    940667 non-null  object 
 1   日期      940667 non-null  int64  
 2   收入类型    940667 non-null  int64  
 3   收入额     940667 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 35.9+ MB


In [179]:
# 定义entropy函数计算
def entropy(x):
    return -(x['p']*np.log2(x['p'] + 1e-5)).sum()

# 计算收入熵
df_entropy = df1.merge(df2, on=['证券代码','日期'], how='left')
df_entropy['p'] = df_entropy.groupby(['证券代码','日期'])['收入额'].apply(lambda x:x/x.sum())
res = df_entropy.groupby(['证券代码','日期']).apply(entropy).reset_index()
res.columns = ['证券代码','日期','收入额']

In [180]:
df1['收入熵指标'] = res['收入额']

In [181]:
df1.head()

,证券代码,日期,收入熵指标
0,#000007,2014,4.429351
1,#000403,2015,4.025647
2,#000408,2016,4.065978
3,#000408,2017,-0.000000
4,#000426,2015,4.448961


In [177]:
res.head()

,证券代码,日期,0
0,#000007,2014,4.429351
1,#000403,2015,4.025647
2,#000408,2016,4.065978
3,#000408,2017,-0.000000
4,#000426,2015,4.448961


## 练习二

In [182]:
team_csv = pd.read_excel('组队信息汇总表Pandas.xls')

In [183]:
# 所在群无用，删除
team_csv.drop(columns='所在群', inplace=True)
team_csv.head()

,队伍名称,队长编号,队长_群昵称,队员1 编号,队员_群昵称,队员2 编号,队员_群昵称.1,队员3 编号,队员_群昵称.2,队员4 编号,...,队员6 编号,队员_群昵称.5,队员7 编号,队员_群昵称.6,队员8 编号,队员_群昵称.7,队员9 编号,队员_群昵称.8,队员10编号,队员_群昵称.9
0,你说的都对队,5,山枫叶纷飞,6,蔡,7.0,安慕希,8.0,信仰,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,熊猫人,175,鱼呲呲,44,Heaven,37.0,吕青,50.0,余柳成荫,82.0,...,25.0,Never say never,55.0,K,120.0,Y.,28.0,X.Y.Q,151.0,swrong
2,中国移不动,107,Y's,124,🥕,75.0,Vito,146.0,张小五,186.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,panda,11,太下真君,35,柚子,108.0,My,42.0,星星点灯,45.0,...,157.0,Zys,158.0,不器,102.0,嘉平佑染,NaN,NaN,NaN,NaN
4,一路向北,13,黄元帅,15,化,16.0,未期,18.0,太陽光下,19.0,...,23.0,🚀,169.0,听风,189.0,Cappuccino,NaN,NaN,NaN,NaN


In [185]:
# 为了进行长宽表转换，需要先修改列名
# 因为之后“是否队长”一列取1表示队长，这里干脆把队长列的名称后缀全改为1，省去之后的转换
# 这里不知道怎么修改队员的列名，那只能用笨办法了
team_csv.columns = ['队伍名称','编号_1','昵称_1','编号_2','昵称_2','编号_3','昵称_3','编号_4','昵称_4','编号_5','昵称_5','编号_6','昵称_6','编号_7','昵称_7','编号_8','昵称_8','编号_9','昵称_9','编号_10','昵称_10','编号_11','昵称_11']

In [211]:
team = pd.wide_to_long(team_csv.reset_index(),
                       stubnames = ['昵称','编号'],
                       i = ['index','队伍名称'],
                       j = '是否队长',
                       sep='_',
                       suffix='.+').dropna().reset_index().drop(columns='index')

In [212]:
# 处理一下是否队长的数据，以及将编号转换为int
team['编号'] = team['编号'].astype(int)
team.loc[team['是否队长']>1,'是否队长']=0

In [220]:
team.head(7)

,队伍名称,是否队长,昵称,编号
0,你说的都对队,1,山枫叶纷飞,5
1,你说的都对队,0,蔡,6
2,你说的都对队,0,安慕希,7
3,你说的都对队,0,信仰,8
4,你说的都对队,0,biubiu🙈🙈,20
5,熊猫人,1,鱼呲呲,175
6,熊猫人,0,Heaven,44
7,熊猫人,0,吕青,37
8,熊猫人,0,余柳成荫,50
9,熊猫人,0,Kuroe,82


,队伍名称,是否队长,昵称,编号
0,你说的都对队,1,山枫叶纷飞,5
1,你说的都对队,0,蔡,6
2,你说的都对队,0,安慕希,7
3,你说的都对队,0,信仰,8
4,你说的都对队,0,biubiu🙈🙈,20
...,...,...,...,...
141,七星联盟,0,Daisy,63
142,七星联盟,0,One Better,131
143,七星联盟,0,rain,112
144,应如是,1,思无邪,54


## 任务三

In [151]:
county = pd.read_csv('county_population.csv')
vote = pd.read_csv('president_county_candidate.csv')

In [107]:
county.head()

,US County,Population
0,".Autauga County, Alabama",55869
1,".Baldwin County, Alabama",223234
2,".Barbour County, Alabama",24686
3,".Bibb County, Alabama",22394
4,".Blount County, Alabama",57826


In [108]:
vote.head()

,state,county,candidate,party,total_votes,won
0,Delaware,Kent County,Joe Biden,DEM,44552,True
1,Delaware,Kent County,Donald Trump,REP,41009,False
2,Delaware,Kent County,Jo Jorgensen,LIB,1044,False
3,Delaware,Kent County,Howie Hawkins,GRN,420,False
4,Delaware,New Castle County,Joe Biden,DEM,195034,True


In [152]:
#Q1 有多少县满足总投票数超过县人口数的一半

# 首先进行数据格式转换，分隔的时候注意中间有一个几乎看不见的 hhhh
county[['county','state']]=county['US County'].str.split(', ',expand=True)
county['county'] = county['county'].str[1:]
county.drop(columns='US County',inplace=True)

In [153]:
vote_count = vote.groupby(['county','state'])['total_votes'].sum().reset_index()
# 下面这一步发现两个表之间的state和county并不能完全对上，而且还存在有nan，因此用inner进行merge
# res = vote_count.merge(county,on=['county','state'],how='left')
res = vote_count.merge(county,on=['county','state'],how='inner')

# 超出1/2的部分
res_over=res[res['total_votes']>(res['Population']/2)]
res_over.shape[0]

1434

In [113]:
county.head()

,Population,county,state
0,55869,Autauga County,Alabama
1,223234,Baldwin County,Alabama
2,24686,Barbour County,Alabama
3,22394,Bibb County,Alabama
4,57826,Blount County,Alabama


In [137]:
# Q2 把州（state）作为行索引，把投票候选人作为列名，列名的顺序按照候选人在全美的总票数由高到低排序，行列对应的元素为该候选人在该州获得的总票数
vote_by_state = vote.pivot_table(index= 'state',
                                 columns = 'candidate',
                                 values = 'total_votes',
                                 aggfunc = 'sum')
vote_by_state.fillna(0,inplace=True)
vote_by_state.loc['sum'] = vote_by_state.sum(0)
vote_by_state_sorted = vote_by_state.sort_values(by='sum',axis=1,ascending=False)
vote_by_state_sorted.head()


candidate,Joe Biden,Donald Trump,Jo Jorgensen,Howie Hawkins,Write-ins,Rocky De La Fuente,Gloria La Riva,Kanye West,Don Blankenship,Brock Pierce,...,Tom Hoefling,Ricki Sue King,Princess Jacob-Fambro,Blake Huber,Richard Duncan,Joseph Kishore,Jordan Scott,Gary Swing,Keith McCormic,Zachary Scalf
state,,,,,,,,,,,,,,,,,,,,,
Alabama,849648.0,1441168.0,25176.0,0.0,7312.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alaska,153405.0,189892.0,8896.0,0.0,34210.0,318.0,0.0,0.0,1127.0,825.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Arizona,1672143.0,1661686.0,51465.0,0.0,2032.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Arkansas,423932.0,760647.0,13133.0,2980.0,0.0,1321.0,1336.0,4099.0,2108.0,2141.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
California,11109764.0,6005961.0,187885.0,81025.0,80.0,60155.0,51036.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [150]:
# Q3 每一个州下设若干县，定义拜登在该县的得票率减去川普在该县的得票率为该县的BT指标，若某个州所有县BT指标的中位数大于0，则称该州为Biden State，请找出所有的Biden State
vote['pre'] = vote.groupby(['state','county'])['total_votes'].apply(lambda x:x/x.sum())
vote_by_county = vote.pivot(index= ['state','county'],
                            columns = 'candidate',
                            values = 'pre',
                            )
vote_by_county['BT'] = vote_by_county['Joe Biden'] - vote_by_county['Donald Trump']
BT_state = vote_by_county.groupby('state')['BT'].apply(lambda x:x.median()>0).reset_index()
BT_state[BT_state.BT == True]


,state,BT
4,California,True
6,Connecticut,True
7,Delaware,True
8,District of Columbia,True
11,Hawaii,True
21,Massachusetts,True
30,New Jersey,True
39,Rhode Island,True
45,Vermont,True
